In [15]:
import pandas as pd
import numpy as np
from sklearn.model_selection import GridSearchCV,train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LinearRegression

# Lectura de los datos de train

In [16]:
path = 'continuous_train.csv'
data = pd.read_csv(path, sep = ',', na_filter = False)
data = data.set_index('id')

X = data.drop(columns = ['y']).copy()
y_train = data[['y']].copy()


standarscaler = StandardScaler()
X_trained_scaled = standarscaler.fit_transform(X)

X_trained_scaled = pd.DataFrame(X_trained_scaled, index=X.index, columns=X.columns.values)

KeyError: "None of ['id'] are in the columns"

In [4]:
X_trained_scaled.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 150 entries, 1 to 150
Data columns (total 33 columns):
y      150 non-null float64
x1     150 non-null float64
x2     150 non-null float64
x3     150 non-null int64
x4     150 non-null int64
x5     150 non-null int64
x6     150 non-null int64
x7     150 non-null int64
x8     150 non-null float64
x9     150 non-null int64
x10    150 non-null float64
x11    150 non-null float64
x12    150 non-null float64
x13    150 non-null int64
x14    150 non-null int64
x15    150 non-null int64
x16    150 non-null int64
x17    150 non-null int64
x18    150 non-null int64
x19    150 non-null int64
x20    150 non-null int64
x21    150 non-null float64
x22    150 non-null float64
x23    150 non-null float64
x24    150 non-null float64
x25    150 non-null int64
x26    150 non-null int64
x27    150 non-null float64
x28    150 non-null int64
x29    150 non-null int64
x30    150 non-null int64
x31    150 non-null float64
x32    150 non-null float64
dtypes: f

# Lectura Datos de Test

In [8]:
path = 'continuous_test.csv'
test = pd.read_csv(path, sep = ',', na_filter = False)
test = test.set_index('id')

X_test = test.drop(columns = ['y']).copy()
y_test = test[['y']].copy()

X_test_scaled = standarscaler.transform(X_test)

X_test_scaled = pd.DataFrame(X_test_scaled, index=X_test.index, columns=X_test.columns.values)

In [14]:
from sklearn.feature_selection import RFE
# from sklearn.svm import SVR

rfe_selector = RFE(estimator=LinearRegression(), n_features_to_select=100, step=10, verbose=5)
rfe_selector.fit(X_trained_scaled, y)

NameError: name 'X_norm' is not defined